# Практика

Скоринг учащихся (следующий семестр на основе текущего, модель с учётом дисциплины)

Команда
- Product owner: Антонов Илья
- Scrum-master: Нейман Алексей
- Team: Лебкова Марина, Чвиков Матвей, Махров Матвей, Бобков Егор, Труфманов Михаил

## Загрузка библиотек

In [1]:
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error, r2_score
from catboost import Pool, CatBoost, CatBoostRegressor, cv

## Предобработка данных

In [3]:
pre_data = pd.read_excel('Успеваемость_01.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'Успеваемость_01.xlsx'

In [ ]:
pre_data.to_csv('/content/new_data.csv', index=False)

In [ ]:
df_new = pd.read_csv('new_data.csv')

In [ ]:
df_new.head(1)

In [ ]:
df_new = df_new.replace({'I полугодие': 1, 'II полугодие': 2})

#### Удаление данных

Удаление учебных годов (практически) без оценок

In [ ]:
years_to_exclude = ['2023 - 2024', '2024 - 2025', '2025 - 2026', '2026 - 2027', '2027 - 2028']

df_new = df_new[~df_new['Учебный год'].isin(years_to_exclude)].copy()

In [ ]:
df_new = df_new[~df_new['Учебная группа'].str.contains('22')].copy()

Удаление предметов без оценки

In [ ]:
df_new = df_new.dropna(subset=['Оценка (без пересдач)', 'Оценка (успеваемость)'], how='all')

#### Заполнение пропусков в промежуточной аттестации

при условии, что есть итоговая оценка

In [ ]:
df_new['Оценка (без пересдач)'] = df_new.apply(
    lambda row: 2 if pd.isna(row['Оценка (без пересдач)']) and not pd.isna(row['Оценка (успеваемость)']) else row['Оценка (без пересдач)'],
    axis=1
)

#### Добавление столбца с семестром

In [ ]:
def calculate_semester(row):
    group_year = int(row['Учебная группа'].split('-')[1])
    start_year = int(row['Учебный год'].split(' - ')[0])
    course = (start_year % 100) - group_year + 1
    semester = course * 2 - 1 if row['Полугодие'] == 1 else course * 2
    return semester

df_new['Семестр'] = df_new.apply(calculate_semester, axis=1)

In [ ]:
df_new['Программа'] = df_new['Учебная группа'].str.split('-').str[0]

In [ ]:
df_new.head(2)

In [ ]:
disciplines = len(df_new['Дисциплина'].unique())
print(f'Количество уникальных дисциплин: {disciplines}')

In [ ]:
programs = len(df_new['Программа'].unique())
print(f'Количество уникальных программ: {programs}')

In [ ]:
df = df_new
df = df.drop(columns=['Номер ЛД', 'Учебная группа', 'Уровень подготовки', 'Учебный год', 'Полугодие', 'Специальность/направление'])

In [ ]:
df.head(1)

#### Кодирование оценок

In [ ]:
df.replace({'зачтено': 5, 'Отлично': 5,
            'Хорошо': 4,
            'Удовлетворительно': 3,
            'Неудовлетворительно': 2, 'Неявка': 2, 'не зачтено': 2, 'Не допущен': 2,
            'Неявка по ув.причине': 0
            },
           inplace=True)

Так как по ТЗ все оставшиеся null у студента это 2, то заменим их:

In [ ]:
df.fillna(2, inplace=True)

In [ ]:
df.head(2)

#### Создание столбцов для каждого направления

In [ ]:
df_programs = pd.get_dummies(df, columns=['Программа'], prefix='', prefix_sep='')
df_programs = df_programs.drop(columns=['Дисциплина', 'Оценка (без пересдач)', 'Оценка (успеваемость)'])
df_programs = df_programs.astype({col: int for col in df_programs.columns[1:]})
df_programs = df_programs.drop_duplicates()

In [ ]:
df_programs.head()

#### Создание столбцов для каждой дисциплины

Сопоставление промежуточной оценки соответствующей дисциплине

In [ ]:
pivot_df = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (без пересдач)',
                          aggfunc='first'
                          ).reset_index()

pivot_df.columns.name = None
pivot_df.columns = [str(col) for col in pivot_df.columns]

Заполнение отсутствующих предметов нулями

In [ ]:
pivot_df.fillna(0, inplace=True)

In [ ]:
pivot_df = pivot_df.astype({col: int for col in pivot_df.columns[1:]})

In [ ]:
pivot_df.head(2)

#### Подсчёт количества двоек в каждом семестре

In [ ]:
df = pivot_df
df.head(7)

In [ ]:
grades_columns = df.drop(['hash', 'Семестр'], axis=1)

In [ ]:
df['count_2'] = (df == 2).sum(axis=1)
df.head(7)

In [ ]:
# Смещение количества двоек наверх на одну строку для каждого студента
df['next_sem_debts'] = df['count_2'].shift(-1).copy()
# Если встретился следующий студент - последнему известному семестру текущего
# студента соответсвует данное количество долгов без смещения
df['next_sem_debts'] = np.where(df['hash'] != df['hash'].shift(-1), df['count_2'], df['next_sem_debts'])
df['next_sem_debts'].fillna(df['count_2'], inplace=True)
df = df.drop('count_2', axis=1)

In [ ]:
df['next_sem_debts'] = df['next_sem_debts'].astype(int)

In [ ]:
df

#### Подсчёт количества каждого вида оценок

для экспорта в другую модель

In [ ]:
subjects = df.apply(pd.Series.value_counts, axis=1)[[2, 3, 4, 5]].fillna(0)
subjects = subjects.astype(int)

#subjects['total'] = subjects[[2, 3, 4, 5]].sum(axis=1)
#subjects['total'] = pd.to_numeric(subjects['total'], errors='coerce').fillna(0)
subjects.head()

In [ ]:
df.head()

#### Смещение столбцов

In [ ]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
print(cols)

In [ ]:
df = df[cols]

In [ ]:
lessons = df.columns.to_list()[2:]
print(lessons)

In [ ]:
df.head(3)

In [ ]:
df = df.astype({col: int for col in df.columns[2:]})

In [ ]:
df.head(3)

#### Объединение таблицы оценок с таблицей направлений

In [ ]:
df_programs.head(1)

In [ ]:
df_merged = pd.merge(df, df_programs, on=['hash', 'Семестр'], how='inner')

In [ ]:
df_merged.head()

### Обработка данных для другой модели, не учитывающей дисциплины

In [ ]:
column_list = df_merged.columns.to_list()
column_list = column_list[:3] + column_list[-19:]
print(column_list)

In [ ]:
df_merged_cnt = df_merged[column_list]
df_merged_cnt = subjects.join(df_merged_cnt, how='inner')
df_merged_cnt.head(2)

In [ ]:
df_merged_cnt.to_csv('num_of_debts.csv', index=False)

## Загрузка данных в модель

In [ ]:
results = df_merged['next_sem_debts'].to_numpy()
print(f'Минимальное количество двоек: {results.min()}')
print(f'Максимальное количество двоек: {results.max()}')
print(f'Среднее количество двоек: {results.mean():.3f}')

In [ ]:
df_merged = df_merged.drop('hash', axis=1)

In [ ]:
df_merged.head(2)

#### Разделение на трейн-тест

In [ ]:
X = df_merged.drop('next_sem_debts', axis=1)
X = X.round().astype(int)
y = df_merged['next_sem_debts']
y = y.astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
train_pool = Pool(X_train,
                  label=y_train)

test_pool = Pool(X_test,
                 label=y_test)

#### Подбор гиперпараметров

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score


def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_seed': 42,
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'loss_function': 'RMSE',
        'eval_metric': 'R2',
    }
    model = CatBoostRegressor(**params, verbose=False)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best trial:')
best_trial = study.best_trial
print(f'  Value: {best_trial.value:.4f}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
model = CatBoostRegressor(custom_metric=['R2'], iterations=1000, learning_rate=0.2)
model.fit(train_pool, eval_set=[test_pool], verbose=200)

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae:.4f}')
print(f'R2: {r2:.4f}')

## Класс студента

In [ ]:
import json
with open('lessons_next.json', 'w', encoding='utf-8') as file:
  json.dump(lessons, file, ensure_ascii=False)

In [ ]:
lessons = lessons
# в приложении все дисциплины загружаются из json файла
programs = df_programs.columns.to_list()[2:]

class NewStudent:
  def __init__(self):
        self.lessons = lessons
        self.programs = programs
        self.data = {key: np.NaN for key in list(self.lessons) + list(self.programs)}


  def add_score(self, key, score):
    if key in self.data:
      self.data[key] = score


  def load_data_from_dict(self, data_dict):
      for lesson, score in data_dict.items():
          if lesson in self.data:
              self.data[lesson] = score

  def data_processing(self):
    student_data = np.array(self.prepare_data_for_prediction()).reshape(1, -1)
    student_data = np.nan_to_num(student_data, nan=0, copy=True)
    student_data = student_data.astype(int)
    return student_data

  def prepare_data_for_prediction(self):
    return [self.data[key] for key in self.data]

In [ ]:
student = NewStudent()
student.add_score('БПМ', 1)
student.add_score('Семестр', 1)
student.add_score('Введение в специальность', 5)
student.add_score('Вычислительные машины, сети и системы', 5)
student.add_score('Иностранный язык', 5)
student.add_score('Математика', 5)
student.add_score('Программирование и алгоритмизация', 5)
student.add_score('Физика', 5)
student.add_score('Физическая культура и спорт', 5)
student.add_score('Объектно-ориентированное программирование', 5)

Предскажем долю двоек в следующем семестре

In [ ]:
res = model.predict(student.data_processing()).round().astype(int)
print(f'Количество двоек в следующем семестре: {res[0]}')

## Экспорт модели

In [ ]:
import pickle

with open('scoring_next_sem.pkl', 'wb') as file:
  pickle.dump(model, file)